<a href="https://colab.research.google.com/github/NebojsaIvancevic/CNN-Dog-Breed-Classifier/blob/main/dogbreed_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources


> [Deep learning with Python, TensorFlow and Keras](https://pythonprogramming.net/)


> [Dog breed classifier transfer learning](https://towardsdatascience.com/dog-breed-classification-hands-on-approach-b5e4f88c333e)

> [Best pre trained models](https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/)

> [Basics of machine learning image techniques](https://iq.opengenus.org/basics-of-machine-learning-image-classification-techniques/)











In [1]:
#Create a dog breed classifier using convolutional neural networks and a pretrained model

We are using the stanford dogs dataset from kaggle
We are using colab provided gpus so we need to install required libraries.

In [3]:
import tensorflow as tf
# tf.__version__
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import pickle
import cv2
from glob import glob
import numpy as np
from keras.preprocessing.image import load_img
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

Also we are fetching data directly from the google drive.

Defined a function for building our model using mobilenet

In [5]:
def build_model(size, num_classes):
  inputs = Input((size,size,3))
  backbone = MobileNetV2(input_tensor = inputs, include_top=False, weights = "imagenet")
  backbone.trainable = True
  x = backbone.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation="relu")(x)
  x = Dense(num_classes, activation="softmax")(x)

  model = tf.keras.Model(inputs, x)
  return model


In [6]:
def read_image(path, size):
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  image = cv2.resize(image, (size,size))
  image = image / 255.0
  image = image.astype(np.float32)
  return image

In [7]:
def parse_data(x,y):
  x = x.decode()

  num_class = 120
  size = 224

  image = read_image(x,size)
  label = [0] * num_class
  label[y] = 1
  label = np.array(label)
  label = label.astype(np.int32)
  
  return image, label

In [8]:
def tf_parse(x,y):
  x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
  x.set_shape((224, 224, 3))
  y.set_shape((120))
  return x, y
  

In [9]:
def tf_dataset(x,y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset

In [11]:
if __name__ == "__main__":
  train_path = "/content/drive/MyDrive/Datasets/dataset#6dogbreedclassification/train"
  test_path = "/content/drive/MyDrive/Datasets/dataset#6dogbreedclassification/test"
  labels_path = "/content/drive/MyDrive/Datasets/dataset#6dogbreedclassification/labels.csv"

  labels_df = pd.read_csv(labels_path)
  labels.head()
  breed = labels_df["breed"].unique()
  print("Number of breeds: ", len(breed))

  breed2id = {name: i for i, name in enumerate(breed)}

  ids = glbo(train_path)
  labels = []

  for image_id in ids:
    image_id = image_id.split("\\")[-1].split(".")[0]
    breed_name = list(labels_df[labels_df.id == image_id]["breed"])(0)
    breed_idx = breed2id[breed_name]
    labels.append(breed_idx)
  
  ids = ids[:1000]
  labels = labels[:1000]


  ##Splitting data set
  train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42) ##test size 20%
  train_y, valid_y = train_test_split(labels, test_size = 0.2, random_state=42)



  ##hyper parameters
  size = 224
  num_classes = 120
  lr = 1e-4 #0.00001
  batch = 32
  epochs = 20

  ## Building the model

  model = build_model(size, num_classes)
  opt = tf.keras.optimizers.Adam(lr=lr)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])


  ## Dataset

  train_dataset = tf_dataset(train_x, train_y, batch = batch)
  valid_dataset = tf_dataset(valid_x, valid_y, batch = batch)

  ## Training

  callbacks = [
               ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
               ReduceLROnPlateau(factor = 0.2, patience = 5, min_lr = 1e-6)
  ]

  train_steps = (len(train_x)//batch) + 1
  valid_steps = (len(valid_x)//batch) + 1
  model.fit(train_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            validation_data=valid_dataset,
            epochs=epochs,
            callbacks=callbacks)

FileNotFoundError: ignored